In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_iitm_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13,78,0.0,0.6908,Positive,5
1,1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0,0,0.0,0.7508,Positive,5
2,2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0,0,0.0,0.7430,Positive,5
3,3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0,0,0.0,-0.4019,Negative,2
4,4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0,13,1.0,0.6124,Positive,5
5,5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2,12,0.0,0.4939,Positive,4
6,6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0,1,0.0,0.0000,Neutral,3
7,7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5,8,0.0,0.4215,Positive,4
8,8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0,25,0.0,0.0000,Neutral,3
9,9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0,1,0.0,0.4019,Positive,4


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,Interesting visit to Sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13.0,78.0,0.0,0.6908,Positive,5,0.002860,0.008442
1,@iitmadras This research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0.0,0.0,0.0,0.7508,Positive,5,0.000000,0.000000
2,@iitmadras You are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0.0,0.0,0.0,0.7430,Positive,5,0.000000,0.000000
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0.0,0.0,0.0,-0.4019,Negative,2,0.000000,0.000000
4,Centre for Social Innovation and Entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0.0,13.0,1.0,0.6124,Positive,5,0.000000,0.001407
5,.@iitmadras hosts 2nd edition of Carbon Zero C...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2.0,12.0,0.0,0.4939,Positive,4,0.000440,0.001299
6,IIT Madras hosted valedictory function; Comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0.0,1.0,0.0,0.0000,Neutral,3,0.000000,0.000108
7,Over 350 students from rural pockets of TN wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5.0,8.0,0.0,0.4215,Positive,4,0.001100,0.000866
8,A panel discussion on the DRAFT NEW EDUCATION ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0.0,25.0,0.0,0.0000,Neutral,3,0.000000,0.002706
9,"Ah, you let the cat out of the bag and Yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0.0,1.0,0.0,0.4019,Positive,4,0.000000,0.000108


In [6]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [7]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [8]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [9]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [10]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,interesting visit to sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13.0,78.0,0.0,0.6908,Positive,5,0.002860,0.008442,0.509279,Interesting visit to Sotacarbo centre of excel...,"[interesting, visit, sotacarbo, centre, excell..."
1,@iitmadras this research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0.0,0.0,0.0,0.7508,Positive,5,0.000000,0.000000,0.000000,@iitmadras This research can save $300 billion...,"[iitmadras, research, save, 300, billion, per,..."
2,@iitmadras you are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0.0,0.0,0.0,0.7430,Positive,5,0.000000,0.000000,0.000000,@iitmadras You are reliable technology institu...,"[iitmadras, reliable, technology, institute, i..."
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0.0,0.0,0.0,-0.4019,Negative,2,0.000000,0.000000,0.000000,#iitmadras hosts 2nd edition of #carbonzerocha...,"[iitmadras, hosts, 2nd, edition, carbonzerocha..."
4,centre for social innovation and entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0.0,13.0,1.0,0.6124,Positive,5,0.000000,0.001407,0.056277,Centre for Social Innovation and Entrepreneurs...,"[centre, social, innovation, entrepreneurship,..."
5,.@iitmadras hosts 2nd edition of carbon zero c...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2.0,12.0,0.0,0.4939,Positive,4,0.000440,0.001299,0.078351,.@iitmadras hosts 2nd edition of Carbon Zero C...,"[iitmadras, hosts, 2nd, edition, carbon, zero,..."
6,iit madras hosted valedictory function; comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0.0,1.0,0.0,0.0000,Neutral,3,0.000000,0.000108,0.004329,IIT Madras hosted valedictory function Complet...,"[iit, madras, hosted, valedictory, function, c..."
7,over 350 students from rural pockets of tn wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5.0,8.0,0.0,0.4215,Positive,4,0.001100,0.000866,0.100639,Over 350 students from rural pockets of TN wit...,"[350, students, rural, pockets, tn, full, aspi..."
8,a panel discussion on the draft new education ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0.0,25.0,0.0,0.0000,Neutral,3,0.000000,0.002706,0.108225,A panel discussion on the DRAFT NEW EDUCATION ...,"[panel, discussion, draft, new, education, pol..."
9,"ah, you let the cat out of the bag and yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0.0,1.0,0.0,0.4019,Positive,4,0.000000,0.000108,0.004329,"Ah you let the cat out of the bag and Yes,","[ah, let, cat, bag, yes]"


In [11]:
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,interesting visit to sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13.0,78.0,0.0,0.6908,Positive,5,0.002860,0.008442,0.509279,Interesting visit to Sotacarbo centre of excel...,"[interesting, visit, sotacarbo, centre, excell...","[(interesting, VBG), (visit, NN), (sotacarbo, ..."
1,@iitmadras this research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0.0,0.0,0.0,0.7508,Positive,5,0.000000,0.000000,0.000000,@iitmadras This research can save $300 billion...,"[iitmadras, research, save, 300, billion, per,...","[(iitmadras, NNS), (research, NN), (save, VBP)..."
2,@iitmadras you are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0.0,0.0,0.0,0.7430,Positive,5,0.000000,0.000000,0.000000,@iitmadras You are reliable technology institu...,"[iitmadras, reliable, technology, institute, i...","[(iitmadras, NNS), (reliable, JJ), (technology..."
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0.0,0.0,0.0,-0.4019,Negative,2,0.000000,0.000000,0.000000,#iitmadras hosts 2nd edition of #carbonzerocha...,"[iitmadras, hosts, 2nd, edition, carbonzerocha...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed..."
4,centre for social innovation and entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0.0,13.0,1.0,0.6124,Positive,5,0.000000,0.001407,0.056277,Centre for Social Innovation and Entrepreneurs...,"[centre, social, innovation, entrepreneurship,...","[(centre, JJ), (social, JJ), (innovation, NN),..."
5,.@iitmadras hosts 2nd edition of carbon zero c...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2.0,12.0,0.0,0.4939,Positive,4,0.000440,0.001299,0.078351,.@iitmadras hosts 2nd edition of Carbon Zero C...,"[iitmadras, hosts, 2nd, edition, carbon, zero,...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed..."
6,iit madras hosted valedictory function; comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0.0,1.0,0.0,0.0000,Neutral,3,0.000000,0.000108,0.004329,IIT Madras hosted valedictory function Complet...,"[iit, madras, hosted, valedictory, function, c...","[(iit, NN), (madras, NNS), (hosted, VBD), (val..."
7,over 350 students from rural pockets of tn wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5.0,8.0,0.0,0.4215,Positive,4,0.001100,0.000866,0.100639,Over 350 students from rural pockets of TN wit...,"[350, students, rural, pockets, tn, full, aspi...","[(350, CD), (students, NNS), (rural, JJ), (poc..."
8,a panel discussion on the draft new education ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0.0,25.0,0.0,0.0000,Neutral,3,0.000000,0.002706,0.108225,A panel discussion on the DRAFT NEW EDUCATION ...,"[panel, discussion, draft, new, education, pol...","[(panel, NN), (discussion, NN), (draft, NN), (..."
9,"ah, you let the cat out of the bag and yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0.0,1.0,0.0,0.4019,Positive,4,0.000000,0.000108,0.004329,"Ah you let the cat out of the bag and Yes,","[ah, let, cat, bag, yes]","[(ah, JJ), (let, VB), (cat, JJ), (bag, VB), (y..."


In [12]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,interesting visit to sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13.0,78.0,0.0,0.6908,Positive,5,0.002860,0.008442,0.509279,Interesting visit to Sotacarbo centre of excel...,"[interesting, visit, sotacarbo, centre, excell...","[(interesting, VBG), (visit, NN), (sotacarbo, ...","[visit, sotacarbo, centre, excellence, carboni..."
1,@iitmadras this research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0.0,0.0,0.0,0.7508,Positive,5,0.000000,0.000000,0.000000,@iitmadras This research can save $300 billion...,"[iitmadras, research, save, 300, billion, per,...","[(iitmadras, NNS), (research, NN), (save, VBP)...","[iitmadras, research, year, india, climate, ch..."
2,@iitmadras you are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0.0,0.0,0.0,0.7430,Positive,5,0.000000,0.000000,0.000000,@iitmadras You are reliable technology institu...,"[iitmadras, reliable, technology, institute, i...","[(iitmadras, NNS), (reliable, JJ), (technology...","[iitmadras, technology, institute, india, worl..."
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0.0,0.0,0.0,-0.4019,Negative,2,0.000000,0.000000,0.000000,#iitmadras hosts 2nd edition of #carbonzerocha...,"[iitmadras, hosts, 2nd, edition, carbonzerocha...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed...","[hosts, edition, carbonzerochallenge, tackle, ..."
4,centre for social innovation and entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0.0,13.0,1.0,0.6124,Positive,5,0.000000,0.001407,0.056277,Centre for Social Innovation and Entrepreneurs...,"[centre, social, innovation, entrepreneurship,...","[(centre, JJ), (social, JJ), (innovation, NN),...","[innovation, entrepreneurship, madras, experts..."
5,.@iitmadras hosts 2nd edition of carbon zero c...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2.0,12.0,0.0,0.4939,Positive,4,0.000440,0.001299,0.078351,.@iitmadras hosts 2nd edition of Carbon Zero C...,"[iitmadras, hosts, 2nd, edition, carbon, zero,...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed...","[hosts, edition, carbon, zero, challenge, boos..."
6,iit madras hosted valedictory function; comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0.0,1.0,0.0,0.0000,Neutral,3,0.000000,0.000108,0.004329,IIT Madras hosted valedictory function Complet...,"[iit, madras, hosted, valedictory, function, c...","[(iit, NN), (madras, NNS), (hosted, VBD), (val...","[iit, madras, function, yr, summer, programme]"
7,over 350 students from rural pockets of tn wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5.0,8.0,0.0,0.4215,Positive,4,0.001100,0.000866,0.100639,Over 350 students from rural pockets of TN wit...,"[350, students, rural, pockets, tn, full, aspi...","[(350, CD), (students, NNS), (rural, JJ), (poc...","[students, pockets, aspirations, iitmrp, stude..."
8,a panel discussion on the draft new education ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0.0,25.0,0.0,0.0000,Neutral,3,0.000000,0.002706,0.108225,A panel discussion on the DRAFT NEW EDUCATION ...,"[panel, discussion, draft, new, education, pol...","[(panel, NN), (discussion, NN), (draft, NN), (...","[panel, discussion, draft, education, policy, ..."
9,"ah, you let the cat out of the bag and yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0.0,1.0,0.0,0.4019,Positive,4,0.000000,0.000108,0.004329,"Ah you let the cat out of the bag and Yes,","[ah, let, cat, bag, yes]","[(ah,

In [13]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,visit
1,sotacarbo
2,centre
3,excellence
4,carbonia
5,program
6,ccus
7,carbon
8,technologies
9,participation


In [14]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,visit,visit,118
1,sotacarbo,sotacarbo,6
2,centre,centre,147
3,excellence,excellence,41
4,carbonia,carbonia,3
5,program,program,217
6,ccus,ccus,3
7,carbon,carbon,38
8,technologies,technologies,166
9,participation,participation,36


In [15]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
71,iit,iit,7568
11,madras,madras,7056
20,iitmadras,iitmadras,6242
18,com,com,5310
17,twitter,twitter,5265
76,students,students,2270
63,pic,pic,1542
23,india,india,1436
36,technology,technology,1230
21,research,research,1102


In [16]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [17]:
l=['makeindiasafer','makemytrip','fathima','caa','fathimalatheef','jamia','makeindiasaferpic']

In [18]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
4008,the cbi will enquire the alleged institutional...,Sharjeel Usmani,1203383099196723200,2019-12-07 18:37:43,[],SharjeelUsmani,2.0,1.0,0.0,-0.4939,Negative,2,0.000440,0.000108,0.030732,The CBI will enquire the alleged institutional...,"[cbi, enquire, alleged, institutional, murder,...","[(cbi, NN), (enquire, NN), (alleged, VBD), (in...","[cbi, enquire, murder, iit, student, fathima, ...",1
4059,i think she is talking about iit madras studen...,तANMAY मALiK™,1202876681402245120,2019-12-06 09:05:23,[],tanmaymalik555,0.0,0.0,0.0,-0.5267,Negative,2,0.000000,0.000000,0.000000,I think she is talking about IIT madras studen...,"[think, talking, iit, madras, student, fathima...","[(think, VB), (talking, VBG), (iit, JJ), (madr...","[madras, student, fathima, latheef, muslim, su...",1
4062,fathima latheef suicide: cbi probe likely with...,S.E. Prabakar,1202857769608417280,2019-12-06 07:50:14,"['suicide', 'Amit', 'IIT', 'Fathima', 'Abdul',...",Seprabakar,1.0,0.0,0.0,-0.8750,Negative,1,0.000220,0.000000,0.013201,Fathima Latheef Suicide CBI Probe Likely With ...,"[fathima, latheef, suicide, cbi, probe, likely...","[(fathima, NN), (latheef, NN), (suicide, NN), ...","[fathima, latheef, suicide, cbi, probe, questi...",1
4064,#icymi a new note surfaces in the #fathimalath...,The Lede,1202851830478204928,2019-12-06 07:26:38,"['ICYMI', 'fathimalatheef']",thelede_in,2.0,4.0,0.0,-0.6705,Negative,1,0.000440,0.000433,0.043719,#ICYMI A new note surfaces in the #fathimalath...,"[icymi, new, note, surfaces, fathimalatheef, s...","[(icymi, JJ), (new, JJ), (note, NN), (surfaces...","[note, surfaces, case, portrait, fathima, way,...",1
4067,iit madras student #fathimalatheef's twin sist...,The New Indian Express,1202833801988182016,2019-12-06 06:15:00,['FathimaLatheef'],NewIndianXpress,0.0,5.0,0.0,0.0000,Neutral,3,0.000000,0.000541,0.021645,IIT Madras student #FathimaLatheef's twin sist...,"[iit, madras, student, fathimalatheef, twin, s...","[(iit, NN), (madras, FW), (student, NN), (fath...","[iit, student, fathimalatheef, twin, sister, m...",1
4068,pm has assured cbi probe into iit madras stude...,Madhyamam,1202830295218573312,2019-12-06 06:01:04,[],madhyamam,0.0,0.0,0.0,-0.4588,Negative,2,0.000000,0.000000,0.000000,PM Has Assured CBI Probe into IIT Madras Stude...,"[pm, assured, cbi, probe, iit, madras, student...","[(pm, NN), (assured, VBD), (cbi, JJ), (probe, ...","[pm, probe, iit, student, fathima, latheef, su...",1
4089,union home minister mr amit shah has given his...,Albert Ancil,1202517808443097088,2019-12-05 09:19:21,[],AlbertAncil,0.0,2.0,0.0,-0.4588,Negative,2,0.000000,0.000216,0.008658,Union Home Minister Mr Amit Shah has given his...,"[union, home, minister, mr, amit, shah, given,...","[(union, NN), (home, NN), (minister, NN), (mr,...","[union, home, minister, mr, amit, word, cbi, p...",1
4138,madras high court has asked why the case of ii...,Maktoob,1201893240128950272,2019-12-03 15:57:33,"['fathimalatheef', 'JusticeForFathimaLatheefht...",MaktoobMedia,1.0,4.0,0.0,-0.8126,Negative,1,0.000220,0.000433,0.030517,Madras High Court has asked why the case of II...,"[madras, high, court, asked, case, iit, madras...","[(madras, NNS), (high, JJ), (court, NN), (aske...","[madras, court, case, iit, student, fathimalat...",1
4316,watch: iit-m cannot give engineering solutions...,Parthiban,1200411942231265280,2019-11-29 13:51:24,"['Mental', 'IIT', 'suicide', 'Fathima', 'aware...",Parthiban2900,0.0,0.0,0.0,-0.8890,Negative,1,0.000000,0.000000,0.000000,Watch IIT-M Cannot Give Engineering Solutions ...,"[watch, iit, give, engineering, solutions, sui...","[(watch, NN), (iit, NN), (give, VBP), (enginee...","[watch, iit, engineering, solutions, health, i...",1
4318,"#watch : no @iitmadras, fitting devices on fan...",The Lede,1200406820075929600,2019-11-29 13:31:03,"['Wat

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']
df_eng1=df[df.engagement==1]
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))
df[df.engagement==1]=df_eng1

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [23]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [24]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,interesting visit to sotacarbo centre of excel...,India in Italy,1138170993191600128,2019-06-10 19:48:05,[],IndiainItaly,13.0,78.0,0.0,0.6908,...,0.002860,0.008442,0.509279,Interesting visit to Sotacarbo centre of excel...,"[interesting, visit, sotacarbo, centre, excell...","[(interesting, VBG), (visit, NN), (sotacarbo, ...","[visit, sotacarbo, centre, excellence, carboni...",0,1,5
1,@iitmadras this research can save $300 billio...,Hari Desai,1138080195263361024,2019-06-10 13:47:17,[],HariDesai12,0.0,0.0,0.0,0.7508,...,0.000000,0.000000,0.000000,@iitmadras This research can save $300 billion...,"[iitmadras, research, save, 300, billion, per,...","[(iitmadras, NNS), (research, NN), (save, VBP)...","[iitmadras, research, year, india, climate, ch...",0,0,5
2,@iitmadras you are reliable technology instit...,Hari Desai,1138079376455548928,2019-06-10 13:44:02,[],HariDesai12,0.0,0.0,0.0,0.7430,...,0.000000,0.000000,0.000000,@iitmadras You are reliable technology institu...,"[iitmadras, reliable, technology, institute, i...","[(iitmadras, NNS), (reliable, JJ), (technology...","[iitmadras, technology, institute, india, worl...",0,0,5
3,#iitmadras hosts 2nd edition of #carbonzerocha...,"IIT Madras, Alumni",1138061888523317248,2019-06-10 12:34:33,"['iitmadras', 'carbonzerochallenge']",iitm_alumni,0.0,0.0,0.0,-0.4019,...,0.000000,0.000000,0.000000,#iitmadras hosts 2nd edition of #carbonzerocha...,"[iitmadras, hosts, 2nd, edition, carbonzerocha...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed...","[hosts, edition, carbonzerochallenge, tackle, ...",0,0,2
4,centre for social innovation and entrepreneurs...,IIT Madras,1138059080268828672,2019-06-10 12:23:23,['FacultyDevelopmentProgram'],iitmadras,0.0,13.0,1.0,0.6124,...,0.000000,0.001407,0.056277,Centre for Social Innovation and Entrepreneurs...,"[centre, social, innovation, entrepreneurship,...","[(centre, JJ), (social, JJ), (innovation, NN),...","[innovation, entrepreneurship, madras, experts...",0,1,3
5,.@iitmadras hosts 2nd edition of carbon zero c...,Education Times,1138027583700783104,2019-06-10 10:18:14,"['sustainablesolutions', 'energy']",educationtimes,2.0,12.0,0.0,0.4939,...,0.000440,0.001299,0.078351,.@iitmadras hosts 2nd edition of Carbon Zero C...,"[iitmadras, hosts, 2nd, edition, carbon, zero,...","[(iitmadras, JJ), (hosts, NNS), (2nd, CD), (ed...","[hosts, edition, carbon, zero, challenge, boos...",0,1,5
6,iit madras hosted valedictory function; comple...,StudyBuzz,1138026135340457984,2019-06-10 10:12:28,[],studybuzztweet,0.0,1.0,0.0,0.0000,...,0.000000,0.000108,0.004329,IIT Madras hosted valedictory function Complet...,"[iit, madras, hosted, valedictory, function, c...","[(iit, NN), (madras, NNS), (hosted, VBD), (val...","[iit, madras, function, yr, summer, programme]",0,0,3
7,over 350 students from rural pockets of tn wit...,IITM Incubation Cell,1138025319988809728,2019-06-10 10:09:14,['StartupJobUtsav'],IITMIC,5.0,8.0,0.0,0.4215,...,0.001100,0.000866,0.100639,Over 350 students from rural pockets of TN wit...,"[350, students, rural, pockets, tn, full, aspi...","[(350, CD), (students, NNS), (rural, JJ), (poc...","[students, pockets, aspirations, iitmrp, stude...",0,1,5
8,a panel discussion on the draft new education ...,evmalert,1138023350851100672,2019-06-10 10:01:24,[],evmalert,0.0,25.0,0.0,0.0000,...,0.000000,0.002706,0.108225,A panel discussion on the DRAFT NEW EDUCATION ...,"[panel, discussion, draft, new, education, pol...","[(panel, NN), (discussion, NN), (draft, NN), (...","[panel, discussion, draft, education, policy, ...",0,1,3
9,"ah, you let the cat out of the bag and yes,",Sairam Radhakrishnan,1137999507810082816,2019-06-10 08:26:40,[],sairamrkn,0.0,1.0,0.0,0.4019,...,0.000000,0.000108,0.004329,"Ah you let the cat out of the bag and Yes,","[ah, let, cat, bag,

In [25]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/IITMWeightedSentiment.csv')